In [2]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

width = 224
height = 224
epochs = 10
batch_size = 32

ruta_train = '/content/drive/My Drive/chest_xray/train/'

train_x = []
train_y = []

# Leemos y procesamos imágenes
for i in os.listdir(ruta_train):
    for j in os.listdir(ruta_train + i):
        img = cv2.imread(ruta_train + i + '/' + j, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue

        resized = cv2.resize(img, (width, height))
        resized = resized / 255.0 # Normalizacion

        # Convertimos imagen de 1 canal (escala de grises) a 3 canales para VGG16
        resized = np.stack((resized,)*3, axis=-1)

        train_x.append(resized)

        if i == "NORMAL":
            train_y.append([0, 1])
        else:
            train_y.append([1, 0])

x_data = np.array(train_x)
y_data = np.array(train_y)

# Dividimos en conjunto de entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Usamos la arquitectura VGG16 preentrenada
def create_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(height, width, 3))

    # "Deshabilitamos" algunas capas para solo entrenar las capas personalizadas
    base_model.trainable = False

    # Capas personalizadas
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(2, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model, base_model

model, base_model = create_model()

# aumentación de datos
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
# Entrenamiento
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_val, y_val),
    epochs=epochs,
    steps_per_epoch=len(x_train) // batch_size,
    verbose=1
)

# Evaluar el modelo entrenado
val_loss, val_acc = model.evaluate(x_val, y_val, verbose=2)
print(f"Validation accuracy: {val_acc}")

# "Habilitamos" las capaz previamente deshabilitadas para fine-tuning
base_model.trainable = True

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-5), metrics=['accuracy'])

# Entrenamos de nuevo con todas las capaz
history_finetune = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_val, y_val),
    epochs=5,
    steps_per_epoch=len(x_train) // batch_size,
    verbose=1
)

# Evaluar después del fine-tuning
val_loss, val_acc = model.evaluate(x_val, y_val, verbose=2)
print(f"Validation accuracy after fine-tuning: {val_acc}")

model.save('modelo_vgg16_finetuned.keras')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/10
67/67 [==============================] - 54s 786ms/step - loss: 0.7885 - accuracy: 0.8235 - val_loss: 0.2008 - val_accuracy: 0.9255
Epoch 2/10
67/67 [==============================] - 52s 770ms/step - loss: 0.2166 - accuracy: 0.9191 - val_loss: 0.1719 - val_accuracy: 0.9348
Epoch 3/10
67/67 [==============================] - 51s 766ms/step - loss: 0.1946 - accuracy: 0.9295 - val_loss: 0.1598 - val_accuracy: 0.9330
Epoch 4/10
67/67 [==============================] - 52s 773ms/step - loss: 0.2156 - accuracy: 0.9262 - val_loss: 0.1826 - val_accuracy: 0.9255
Epoch 5/10
67/67 [==============================] - 52s 768ms/step - loss: 0.1831 - accuracy: 0.9423 - val_loss: 0.1130 - val_accuracy: 0.9590
Epoch 6/10
67/67 [==============================] - 51s 760ms/step - loss: 0.1967 - accurac